In [27]:
import pandas as pd

In [28]:
df = pd.read_csv("Online Sales Data.csv")

In [29]:
df.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


- propiedades  de la tabla 

In [30]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    240 non-null    int64  
 1   Date              240 non-null    object 
 2   Product Category  240 non-null    object 
 3   Product Name      240 non-null    object 
 4   Units Sold        240 non-null    int64  
 5   Unit Price        240 non-null    float64
 6   Total Revenue     240 non-null    float64
 7   Region            240 non-null    object 
 8   Payment Method    240 non-null    object 
dtypes: float64(2), int64(2), object(5)
memory usage: 17.0+ KB


- verificando que cada id sea unico

In [31]:
len(df["Transaction ID"].unique())

240

- verificando valores unicos

In [32]:
df["Product Category"].unique()

array(['Electronics', 'Home Appliances', 'Clothing', 'Books',
       'Beauty Products', 'Sports'], dtype=object)

In [33]:
df["Region"].unique()

array(['North America', 'Europe', 'Asia'], dtype=object)

In [34]:
df["Payment Method"].unique()

array(['Credit Card', 'PayPal', 'Debit Card'], dtype=object)

-verificando si cada producto tiene mas de un valor unitario

In [35]:
dicionario_precios = df.groupby("Product Name")["Unit Price"].unique().apply(list).to_dict()

In [36]:
for key,value in dicionario_precios.items():
    if len(value)>1:
        print(key,value)

Dune by Frank Herbert [np.float64(25.99), np.float64(9.99)]
Garmin Forerunner 945 [np.float64(499.99), np.float64(599.99)]
Keurig K-Elite Coffee Maker [np.float64(189.99), np.float64(169.99)]
The Catcher in the Rye by J.D. Salinger [np.float64(13.99), np.float64(10.99)]
The Girl with the Dragon Tattoo by Stieg Larsson [np.float64(9.99), np.float64(10.99)]


In [37]:
df["Date"] = pd.to_datetime(df["Date"])

In [38]:
df.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


crear dimensiones de Region - Payment Method

In [39]:
def crear_dimension(data,id_name):
    list_keys = []
    value = 1

    for _ in data:
        list_keys.append(value)
        value += 1
    return pd.DataFrame({id_name:list_keys,"values":data})

In [40]:
df_region = crear_dimension(df["Region"].unique(),"id_region")

In [41]:
df_payment = crear_dimension(df["Payment Method"].unique(),"id_payment_method")

In [42]:
df_final = df.merge(df_region,how="left",left_on="Region",right_on="values")

In [43]:
df_final = df_final.merge(df_payment,how="left",left_on="Payment Method",right_on="values")

In [44]:
df_final=df_final[["Transaction ID","Date","Product Category","Product Name","Units Sold","Unit Price","Total Revenue","id_region","id_payment_method"]].copy()

In [45]:
df_final.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,id_region,id_payment_method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,1,1
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,2,2
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,3,3
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,1,1
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,2,2


Conexion con la base de datos


In [9]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv,dotenv_values

In [10]:
load_dotenv()

True

In [11]:
password=os.getenv("POSTGRES_PASS")
user=os.getenv("POSTGRES_USER")
db=os.getenv("POSTGRESS_DB")
url=os.getenv("URL")

In [13]:
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{url}/{db}")

Cargando las tablas con la funcion to_sql

In [ ]:
df_region.to_sql("region",engine,if_exists="replace",index=False)
df_payment.to_sql("payment_method",engine,if_exists="replace",index=False)

In [46]:
columns_name = {"Transaction ID":"id_transaction","Date":"date","Product Category":"product_category",
                "Product Name":"product_name","Units Sold":"unit_sold","Unit Price":"unit_price","Total Revenue":"total_revenue"}

In [47]:
df_final.rename(columns=columns_name,inplace=True)

In [ ]:
df_final.to_sql("sales",engine,if_exists="replace",index=False)